In [ ]:
!pip install torch torchvision ftfy regex tqdm faiss-cpu gradio git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zi56e2h6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zi56e2h6
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import torchvision.transforms as T
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import faiss
import gradio as gr
from tqdm import tqdm
import clip

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Download sample images
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/2/26/YellowLabradorLooking_new.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/b/bc/Juvenile_Ragdoll.jpg",
]

os.makedirs("dataset_images", exist_ok=True)
print("🔽 Downloading sample images...")
for i, url in enumerate(image_urls):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        with open(f"dataset_images/image_{i}.jpg", "wb") as f:
            f.write(response.content)
    except Exception as e:
        print(f"⚠️ Failed to download image: {url} Reason: {e}")
print("✅ Image download complete.")

# Load CLIP model
print("📦 Loading CLIP model...")
model, preprocess = clip.load("ViT-B/32", device=device)
model.eval()

# Feature extraction function using CLIP
def extract_features_clip(image_path):
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.encode_image(image)
    features = features.cpu().numpy().flatten()
    features = features / np.linalg.norm(features)
    return features

# Extract features from dataset images
print("🔍 Extracting features from dataset images...")
dataset_features = []
dataset_image_paths = []
for filename in tqdm(os.listdir("dataset_images")):
    path = os.path.join("dataset_images", filename)
    try:
        features = extract_features_clip(path)
        dataset_features.append(features)
        dataset_image_paths.append(path)
    except Exception as e:
        print(f"Error in {path}: {e}")

dataset_features = np.array(dataset_features).astype("float32")
print(f"✅ Successfully extracted {len(dataset_features)} feature vectors.")

# Build FAISS index
index = faiss.IndexFlatL2(dataset_features.shape[1])
index.add(dataset_features)
print(f"✅ FAISS index created with {len(dataset_features)} images.")

# Function to search for similar images
def search_similar_images(query_img, top_k=4):
    image = preprocess(query_img.convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.encode_image(image)
    features = features.cpu().numpy().flatten()
    features = features / np.linalg.norm(features)
    query_feat = np.array([features]).astype("float32")

    distances, indices = index.search(query_feat, top_k)
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        similarity = 1.0 - min(dist, 1.0)  # Normalize similarity score
        results.append((dataset_image_paths[idx], similarity))
    return results

# Gradio function
def gradio_search(image):
    results = search_similar_images(image)
    images = [Image.open(path) for path, _ in results]
    similarities = {f"{os.path.basename(path)}": round(score, 3) for path, score in results}
    return images, similarities

# Gradio interface
title = "🔍 Image Search Engine with CLIP & FAISS"
description = "Upload an image to find the most visually similar ones using CLIP-based feature extraction."

iface = gr.Interface(
    fn=gradio_search,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Gallery(label="Similar Images", show_label=True, columns=2),
        gr.Label(label="Similarity Score (Closer to 1 is better)")
    ],
    title=title,
    description=description,
    flagging_mode=None,
)

iface.launch(share=True, debug=True)

🔽 Downloading sample images...
✅ Image download complete.
📦 Loading CLIP model...
🔍 Extracting features from dataset images...


100%|██████████| 4/4 [00:00<00:00, 15.89it/s]


✅ Successfully extracted 4 feature vectors.
✅ FAISS index created with 4 images.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c0666a8fc1f1196ff8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
